In [2]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch

from torch.utils.data import Dataset, DataLoader, random_split
from datasets import load_dataset
import pandas as pd
from scripts.pre_rn18 import ResNetFinetuner, eigen_train, CustomDataset
from scripts.datasets import office_home, convert_bytes_to_images
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import numpy as np
import warnings

In [ ]:
ds = load_dataset("flwrlabs/office-home")

class_names, real_data = office_home('real')
print(f"Class names: {class_names}")

In [ ]:
real_data_df = real_data.to_pandas()

# Split the in-distribution data into training (80%), validation (10%), and testing (10%)
train_val_df, test_df = train_test_split(
    real_data_df, 
    test_size=0.2, 
    random_state=42, 
    stratify=real_data_df['label']
)
train_df, val_df = train_test_split(
    train_val_df, 
    test_size=0.125, 
    random_state=42, 
    stratify=train_val_df['label']
)

from datasets import Dataset
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)
test_data = Dataset.from_pandas(test_df)

print(f"In-Distribution Training Data: {len(train_data)}")
print(f"In-Distribution Validation Data: {len(val_data)}")
print(f"In-Distribution Test Data: {len(test_data)}")

In [4]:
train_images = train_data['image']
train_labels = train_data['label']
classes = ds['train'].features['label'].names
num_classes = len(set(classes))

val_images = val_data['image']
val_labels = val_data['label']

test_images = test_data['image']
test_labels = test_data['label']

train_images = convert_bytes_to_images(train_images)
val_images = convert_bytes_to_images(val_images)
test_images = convert_bytes_to_images(test_images)

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))  
])

In [5]:
# Create custom datasets
train_dataset = CustomDataset(images=train_images, labels=train_labels, classes=classes, transform=transform)
val_dataset = CustomDataset(images=val_images, labels=val_labels, classes=classes, transform=transform)
test_dataset = CustomDataset(images=test_images, labels=test_labels, classes=classes, transform=transform)

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model = ResNetFinetuner(num_classes=num_classes)
torch.save(model.state_dict(), 'loss_model/model_bs.pth') #path

learning_rate = 1e-5
epochs = 5

ft_model, trace_bs = eigen_train(model, train_dataloader, num_epochs=epochs)
torch.save(model.state_dict(), 'loss_model/ft_model_bs.pth') #path

In [ ]:
len(trace_bs)

Dropout

In [ ]:
dropout_rate = [0.1, 0.3, 0.5, 0.7]

model_dp = {}
ft_model_dp = {}
trace_dp = {}

for i in dropout_rate:
    # Define models and watchers
    model_dp[i] = ResNetFinetuner(num_classes=num_classes, dropout_rate=i)
    # Save model state
    torch.save(model_dp[i].state_dict(), f'loss_model/model_dp{i}.pth')
    # Train the model
    ft_model_dp[i], trace_dp[i] = eigen_train(model_dp[i], train_dataloader, num_epochs=epochs)

    # Save fine-tuned model state
    torch.save(ft_model_dp[i].state_dict(), f'loss_model/ft_model_dp{i}.pth')

Weight Decay

In [ ]:
weight_decay_values = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3]

model_wd = {}
ft_model_wd = {}
trace_wd = {}

# Dictionaries to store evaluation metrics
test_loss_dict_wd = {}
test_acc_dict_wd = {}
test_ece_dict_wd = {}

for i in weight_decay_values:
    # Define models and watchers
    model_wd[i] = ResNetFinetuner(num_classes=num_classes)

    # Save model state
    torch.save(model_wd[i].state_dict(), f'loss_model/model_wd{i}.pth')


    # Train the model
    ft_model_wd[i], trace_wd[i]= eigen_train(model_wd[i], train_dataloader, num_epochs=epochs, weight_decay=i)

    # Save fine-tuned model state
    torch.save(ft_model_wd[i].state_dict(), f'loss_model/ft_model_wd{i}.pth')

In [10]:
trace_bs = pd.DataFrame([trace_bs])
trace_dp = pd.DataFrame([trace_dp])
trace_wd = pd.DataFrame([trace_wd])

trace_bs.to_csv('loss_model/trace_bs.csv', index=False)
trace_dp.to_csv('loss_model/trace_dp.csv', index=False)
trace_wd.to_csv('loss_model/trace_wd.csv', index=False)